### Baseline Model

Here, we implement bigram Naive Bayes as a baseline model to compare future models to. We choose a bigram model based on the exploratory data analysis indicating the certain phrases are seen more in non drug-drug interaction sentences than in sentences with drug-drug interactions. To normalize occurence count, we adjust frequencies with TF-IDF.

In [1]:
import pandas as pd
import numpy as np
import nltk
import sklearn
import bisect
from scipy import sparse
from sklearn.base import TransformerMixin
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV

#### Baseline Model with Drug Pairs

As each sentence can have multiple drugs mentioned and since we are looking at potential drug pairs that cause DDIs, the same sentences can have multiple instances of DDIs with different drug pairs. For example, a sentence like "It is possible that ketoconazole and itraconazole may inhibit the metabolism of carbamazepine." has three drug pairs to consider: ketoconazole and itraconazole; ketoconazole and carbamazepine; and itraconazole and carbamazepine. Since each entry of the dataset considers a sentence and a drug pair, we use the sentence and drug pair as features for our Naive model. 

In [3]:
colnames = ['sentence_id', 'text', 'pair_id', 'drug1_id', 'drug1', 'drug1_type', 'drug2_id', 'drug2', 'drug2_type', 'ddi', 'ddi_type']
train_data = pd.read_csv('./ddi_train.csv', header = None, names = colnames)
test_data = pd.read_csv('./ddi_test.csv', header = None, names = colnames)

In [4]:
def clean_data(data, test = False):
    '''performs data cleaning steps'''
    data['ddi_type'] = data['ddi_type'].fillna('none')
    data['drug1'] = data['drug1'].map(lambda x: x.lower())
    data['drug2'] = data['drug2'].map(lambda x: x.lower())
    
    ## replace unseen drugs in the test set with '<unknown>'
    if test:
        drug1 = np.unique(train_data['drug1'])
        drug2 = np.unique(train_data['drug2'])
        data['drug1'] = data['drug1'].map(lambda x: '<unknown>' if x not in drug1 else x)
        data['drug2'] = data['drug2'].map(lambda x: '<unknown>' if x not in drug2 else x)

def select_drug1(data):
    return data['drug1']

def select_drug2(data):
    return data['drug2']

def make_sparse(array):
    return sparse.csr_matrix(np.reshape(array, (-1, 1)))

def tokenize(data, tokenizer_function = nltk.word_tokenize):
    return data.apply(lambda row: " ".join(tokenizer_function(row['text'])), axis = 1)

#A custom LabelEncoder is needed because the sklearn pipeline assumes that LabelEncoder is defined to take three
#positional arguments (self, x, y) when it only needs two (self, x). We add a argument for y that defaults to 0.
class MyLabelEncoder(TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.encoder = LabelEncoder(*args, **kwargs)
    def fit(self, x, y = 0):
        self.encoder.fit(x)
        
        #add an <unknown> tag to classes to corrent for unseen label errors in fitting
        classes = self.encoder.classes_.tolist()
        bisect.insort_left(classes, '<unknown>')
        self.classes_ = np.array(classes)
        self.encoder.classes_ = self.classes_
        #print('<unknown>' in self.classes_)
        
        return self
    def transform(self, x, y = 0):
        
        #bisect.insort_left(classes, '<unknown>')
        #self.classes_ = classes
        #self.encoder.classes_ = classes
        return self.encoder.transform(x)
    

drug1_pipeline = Pipeline(steps = [
    ('select', FunctionTransformer(select_drug1, validate = False)),
    ('encode', MyLabelEncoder()),
    ('to_sparse', FunctionTransformer(make_sparse, validate = False))
])

drug2_pipeline = Pipeline(steps = [
    ('select', FunctionTransformer(select_drug2, validate = False)),
    ('encode', MyLabelEncoder()),
    ('to_sparse', FunctionTransformer(make_sparse, validate = False))
])

text_pipeline = Pipeline(steps = [
    ('tokenizer', FunctionTransformer(tokenize, validate = False)),
    ('vectorizer', CountVectorizer(ngram_range = (1, 1)))
])

preprocessing_pipeline = FeatureUnion([('drug1', drug1_pipeline), 
                                       ('drug2', drug2_pipeline), 
                                       ('text', text_pipeline)])

model_pipeline = Pipeline(steps = [('preprocess', preprocessing_pipeline),
                                   ('classifier', MultinomialNB())
])

In [5]:
clean_data(train_data)
clean_data(test_data, test = True)

In [6]:
train_data.head()

,sentence_id,text,pair_id,drug1_id,drug1,drug1_type,drug2_id,drug2,drug2_type,ddi,ddi_type
0,DDI-DrugBank.d519.s3,laboratory tests response to plenaxis should b...,DDI-DrugBank.d519.s3.p0,DDI-DrugBank.d519.s3.e0,plenaxis,brand,DDI-DrugBank.d519.s3.e1,testosterone,drug,False,none
1,DDI-DrugBank.d297.s1,population pharmacokinetic analyses revealed t...,DDI-DrugBank.d297.s1.p0,DDI-DrugBank.d297.s1.e0,mtx,drug,DDI-DrugBank.d297.s1.e1,nsaids,group,False,none
2,DDI-DrugBank.d297.s1,population pharmacokinetic analyses revealed t...,DDI-DrugBank.d297.s1.p1,DDI-DrugBank.d297.s1.e0,mtx,drug,DDI-DrugBank.d297.s1.e2,corticosteroids,group,False,none
3,DDI-DrugBank.d297.s1,population pharmacokinetic analyses revealed t...,DDI-DrugBank.d297.s1.p2,DDI-DrugBank.d297.s1.e0,mtx,drug,DDI-DrugBank.d297.s1.e3,tnf blocking agents,group,False,none
4,DDI-DrugBank.d297.s1,population pharmacokinetic analyses revealed t...,DDI-DrugBank.d297.s1.p3,DDI-DrugBank.d297.s1.e0,mtx,drug,DDI-DrugBank.d297.s1.e4,abatacept,drug,False,none


In [7]:
test_data.head()

,sentence_id,text,pair_id,drug1_id,drug1,drug1_type,drug2_id,drug2,drug2_type,ddi,ddi_type
0,DDI-DrugBank.d610.s0,pharmacokinetic properties of abacavir were no...,DDI-DrugBank.d610.s0.p0,DDI-DrugBank.d610.s0.e0,<unknown>,drug,DDI-DrugBank.d610.s0.e1,lamivudine,drug,False,none
1,DDI-DrugBank.d610.s0,pharmacokinetic properties of abacavir were no...,DDI-DrugBank.d610.s0.p1,DDI-DrugBank.d610.s0.e0,<unknown>,drug,DDI-DrugBank.d610.s0.e2,zidovudine,drug,False,none
2,DDI-DrugBank.d610.s0,pharmacokinetic properties of abacavir were no...,DDI-DrugBank.d610.s0.p2,DDI-DrugBank.d610.s0.e0,<unknown>,drug,DDI-DrugBank.d610.s0.e3,lamivudine,drug,False,none
3,DDI-DrugBank.d610.s0,pharmacokinetic properties of abacavir were no...,DDI-DrugBank.d610.s0.p3,DDI-DrugBank.d610.s0.e0,<unknown>,drug,DDI-DrugBank.d610.s0.e4,zidovudine,drug,False,none
4,DDI-DrugBank.d610.s0,pharmacokinetic properties of abacavir were no...,DDI-DrugBank.d610.s0.p4,DDI-DrugBank.d610.s0.e1,lamivudine,drug,DDI-DrugBank.d610.s0.e2,zidovudine,drug,False,none


In [8]:
y_train_identification = train_data['ddi']
y_train_classification = train_data['ddi_type']
y_test_identification = test_data['ddi']
y_test_classification = test_data['ddi_type']

#### Predict DDI Identification

In [9]:
model_pipeline.fit(train_data, y_train_identification)

Pipeline(steps=[('preprocess', FeatureUnion(n_jobs=1,
       transformer_list=[('drug1', Pipeline(steps=[('select', FunctionTransformer(accept_sparse=False,
          func=<function select_drug1 at 0x00000209F3E71598>,
          inv_kw_args=None, inverse_func=None, kw_args=None, pass_y=False,
          valid...nsformer_weights=None)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [10]:
prediction = model_pipeline.predict(test_data)

In [11]:
accuracy_score(y_test_identification, prediction)

0.4734544184421935

In [12]:
f1_score(y_test_identification, prediction)

0.25167535368577815

In [13]:
print(classification_report(y_test_identification, prediction))

             precision    recall  f1-score   support

      False       0.82      0.46      0.59      4743
       True       0.17      0.52      0.25       983

avg / total       0.71      0.47      0.54      5726



In [14]:
model_pipeline.fit(train_data, y_train_classification)

Pipeline(steps=[('preprocess', FeatureUnion(n_jobs=1,
       transformer_list=[('drug1', Pipeline(steps=[('select', FunctionTransformer(accept_sparse=False,
          func=<function select_drug1 at 0x00000209F3E71598>,
          inv_kw_args=None, inverse_func=None, kw_args=None, pass_y=False,
          valid...nsformer_weights=None)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [15]:
prediction = model_pipeline.predict(test_data)

In [16]:
accuracy_score(y_test_classification, prediction)

0.3010827803003842

In [17]:
f1_score(y_test_classification, prediction, average = 'micro')

0.3010827803003842

In [18]:
print(classification_report(y_test_classification, prediction))

             precision    recall  f1-score   support

     advise       0.10      0.46      0.17       222
     effect       0.12      0.36      0.18       363
        int       0.05      0.54      0.08        96
  mechanism       0.12      0.37      0.18       302
       none       0.87      0.28      0.42      4743

avg / total       0.74      0.30      0.38      5726



The model seems to be best at predicting whether no drug-drug interaction exists. One reason for this is that there is an overwhelming amount of drug pairs that do not describe drug-drug interactions while the rest of the classifications suffer from sparsity issues. This may be aleviated in the future by discarding negative pairs from the initial training set, a strategy commonly used in the literature.

In [19]:
train_data['ddi_type'].value_counts()

none         23846
effect        1690
mechanism     1325
advise         826
int            188
Name: ddi_type, dtype: int64